## AISE4010- Project - Traffic Congestion

### Group 7:
#### Aaron Triguero, Khen Agnes, Shiv Pi Patel



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Normalization
from keras.layers import LSTM, SimpleRNN, Conv1D, GRU
from tensorflow.keras.utils import plot_model

## View raw files for speed, occupancy, travel time

In [2]:
# Load Occupancy dataset
df_o1 = pd.read_csv('occupancy_6005.csv').rename(columns={'value':'Occupancy'})
df_o2 = pd.read_csv('occupancy_t4013.csv').rename(columns={'value':'Occupancy'})

# Set index
df_o1['timestamp'] = pd.to_datetime(df_o1['timestamp'])   # to datetime
df_o2['timestamp'] = pd.to_datetime(df_o2['timestamp'])
df_o1 = df_o1.set_index('timestamp')
df_o2 = df_o2.set_index('timestamp')
print(df_o2.head())

# plot
plt.figure(figsize=(16,8))
plt.subplot(2, 1, 1)
df_o1['Occupancy'].plot(ylabel='average # cars',title='Occupancy')
plt.subplot(2, 1, 2)
df_o2['Occupancy'].plot(ylabel='average # cars')

FileNotFoundError: [Errno 2] No such file or directory: 'occupancy_6005.csv'

In [ ]:
# Load TravelTime dataset
df_t1 = pd.read_csv('TravelTime_387.csv').rename(columns={'value':'Travel Time'})
df_t2 = pd.read_csv('TravelTime_451.csv').rename(columns={'value':'Travel Time'})

# Set index
df_t1['timestamp'] = pd.to_datetime(df_t1['timestamp'])   # to datetime
df_t2['timestamp'] = pd.to_datetime(df_t2['timestamp'])
df_t1 = df_t1.set_index('timestamp')
df_t2 = df_t2.set_index('timestamp')
print(df_t2.head())

# plot
plt.figure(figsize=(16,8))
plt.subplot(2, 1, 1)
df_t1['Travel Time'].plot(ylabel='seconds',title='Travel Time')
plt.subplot(2, 1, 2)
df_t2['Travel Time'].plot(ylabel='seconds')

In [ ]:
# Load speed dataset
df_s1 = pd.read_csv('speed_6005.csv').rename(columns={'value':'Speed'})
df_s2 = pd.read_csv('speed_t4013.csv').rename(columns={'value':'Speed'})
df_s3 = pd.read_csv('speed_7578.csv').rename(columns={'value':'Speed'})

# Set index
df_s1['timestamp'] = pd.to_datetime(df_s1['timestamp'])   # to datetime
df_s2['timestamp'] = pd.to_datetime(df_s2['timestamp'])
df_s3['timestamp'] = pd.to_datetime(df_s3['timestamp'])
df_s1 = df_s1.set_index('timestamp')
df_s2 = df_s2.set_index('timestamp')
df_s3 = df_s3.set_index('timestamp')
print(df_s2.head())

# plot
plt.figure(figsize=(16,8))
plt.subplot(3, 1, 1)
df_s1['Speed'].plot(ylabel='km/h',title='Speed')
plt.subplot(3, 1, 2)
df_s2['Speed'].plot(ylabel='km/h')
plt.subplot(3, 1, 3)
df_s3['Speed'].plot(ylabel='km/h')

## 1. Form Dataset and Preprocess

- notice preprocess choices made, such as resampling

#### Joining, Resampling and Normalizing

In [ ]:
# Dataset for intersection 6005
# data_6005 = df_o1.join(df_t1, how='outer').join(df_s1, how='outer')
# data_6005 = data_6005['2015-09-09':]  # take only data after certain date             # if we wanted to do 2 intersections

# Dataset for intersection t4013
data_t4013 = df_o2.join(df_t2, how='outer').join(df_s2, how='outer')
data = data_t4013['2015-09-09':]  # take only data after certain date



# Plot data before resampling
plt.figure(figsize=(16,8))
plt.subplot(2,1,1)
data['Speed'].plot(title='Intersection_t4013, before and after preprocess')#, ylabel='(different for each variable)')
data['Occupancy'].plot()
data['Travel Time'].plot()


# Data after resampling (using mean) to 5 min intervals
# and handling missing data using _______(interpolation)________
data = data.resample('5min').mean()
data = data.interpolate()

# Normalize each variable using ________(MinMax)_________
scaler = MinMaxScaler()
data['Speed'] = scaler.fit_transform(data[['Speed']])
data['Occupancy'] = scaler.fit_transform(data[['Occupancy']])
data['Travel Time'] = scaler.fit_transform(data[['Travel Time']])


# Plot data after preprocessing
plt.subplot(2,1,2)
data['Speed'].plot()#ylabel='(different for each variable)')
data['Occupancy'].plot()
data['Travel Time'].plot()
plt.legend()
plt.show()


In [ ]:
data.shape    # (2510 timesteps, 3 variables)

#### Components of the time series

In [ ]:
# check seasonality, trends, etc for each variable

#### Splitting train and test sets

In [ ]:
# Sliding window to split data

def sliding_window(data, n):
    # data: original data
    # n: window size
    X, y = [], []
    for i in range(len(data) - n):
        X.append(data[i:(i + n)])
        y.append(data[i + n])
    return np.array(X), np.array(y)


# For n timesteps
n_steps = 12      # each 60 minutes (5 minutes per timestep)

# Each variable
Xo, yo = sliding_window(data['Occupancy'], n_steps)
Xs, ys = sliding_window(data['Speed'], n_steps)
Xt, yt = sliding_window(data['Travel Time'], n_steps)

# Split data into training testing sets
split = int(len(Xo)*0.8)

Xo_train, Xo_test = Xo[:split], Xo[split:]
yo_train, yo_test = yo[:split], yo[split:]

Xs_train, Xs_test = Xs[:split], Xs[split:]
ys_train, ys_test = ys[:split], ys[split:]

Xt_train, Xt_test = Xt[:split], Xt[split:]
yt_train, yt_test = yt[:split], yt[split:]

# Reshape
Xo_train = Xo_train.reshape(Xo_train.shape[0], Xo_train.shape[1], 1)
Xo_test = Xo_test.reshape(Xo_test.shape[0], Xo_test.shape[1], 1)

Xs_train = Xs_train.reshape(Xs_train.shape[0], Xs_train.shape[1], 1)
Xs_test = Xs_test.reshape(Xs_test.shape[0], Xs_test.shape[1], 1)

Xt_train = Xt_train.reshape(Xt_train.shape[0], Xt_train.shape[1], 1)
Xt_test = Xt_test.reshape(Xt_test.shape[0], Xt_test.shape[1], 1)


## 2. Models, Training

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Normalization
from keras.layers import LSTM, SimpleRNN, Conv1D, GRU
from tensorflow.keras.utils import plot_model

from time import time

### CNN Model

In [ ]:
# create models
# CNN model
model_CNN = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(Xo_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

# Optimizer ________ and loss _________
model_CNN.compile(optimizer='adam', loss='mean_squared_error')    # why?

model_CNN_s, model_CNN_o, model_CNN_t= model_CNN, model_CNN, model_CNN

# Choose parameters and train dataset
EPOCHS = 50
BATCH_SIZE = 32



#### Training using CNN models

In [ ]:
# Speed data
start_time = time()
cnn_history_speed = model_CNN_s.fit(Xs_train, ys_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)
cnn_time_speed = time() - start_time


In [ ]:
# Occupancy data
start_time = time()
cnn_history_occupancy = model_CNN_o.fit(Xo_train, yo_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)
cnn_time_occupancy = time() - start_time


In [ ]:
# Travel Time data
start_time = time()
cnn_history_ttime = model_CNN_t.fit(Xt_train, yt_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)
cnn_time_ttime = time() - start_time
